# Two Stream Generation 
### Backgroun + Foreground
#### Reference: MIT Generating Video with Scene Dynamics[http://web.mit.edu/vondrick/tinyvideo/]

In [1]:
from util import *
from util.parser import *
from util.img_kit import *
from util.tf_ops import *
from util.notebook_display import *
from util.numeric_ops import *
from IPython import display
import numpy as np
from scipy import ndimage
from scipy import misc
from os import walk
import os
import tensorflow as tf
from PIL import Image

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['figure.figsize'] = (5.0, 5.0) # set default size of plots
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

In [2]:
img_collections = get_processed_moving_box()
# number of images for each collection
num_per_collection = [x.shape[0] for x in img_collections]
# number of collections
n_collection = len(img_collections)
# total number of images
total_imgs = sum(num_per_collection)
print("\nAfter Augmentation: img_collections has {} collections, {} images in total".format(n_collection, total_imgs))

{'dim': (50, 32, 32), 'images:': 'triangle'}
{'dim': (56, 32, 32), 'images:': 'rectangle'}
{'dim': (56, 32, 32), 'images:': 'trangle-vertical'}
{'dim': (56, 32, 32), 'images:': 'circle-diagnal'}
{'dim': (56, 32, 32), 'images:': 'square-diagnal-2'}
{'dim': (56, 32, 32), 'images:': 'rectangle'}
{'dim': (56, 32, 32), 'images:': 'square-vertical-2'}
{'dim': (56, 32, 32), 'images:': 'trangle-vertical-3'}
{'dim': (56, 32, 32), 'images:': 'trangle-horizontal'}
{'dim': (56, 32, 32), 'images:': 'square-vertical-3'}
{'dim': (56, 32, 32), 'images:': 'rectangle'}
{'dim': (56, 32, 32), 'images:': 'trangle-vertical-2'}
{'dim': (56, 32, 32), 'images:': 'rectangle'}
{'dim': (14, 32, 32), 'images:': 'moving box uniform'}
{'dim': (56, 32, 32), 'images:': 'circle'}
{'dim': (56, 32, 32), 'images:': 'rectangle'}
{'dim': (56, 32, 32), 'images:': 'rectangle'}
{'dim': (56, 32, 32), 'images:': 'rectangle'}
{'dim': (56, 32, 32), 'images:': 'circle-diagnal-2'}
{'dim': (56, 32, 32), 'images:': 'square-vertical'}


In [ ]:
train_ind_collection, val_ind_collection = zip(*[split_train_dev(x) for x in img_collections])

In [ ]:
def sample(batch_size = 8, train=True, gap = 1):
    # get average number of training for each class
    avg_num_per_class = int(np.ceil(batch_size/n_collection))
    # before-index for each class
    before_ind = []
    for i in range(n_collection):
        data = train_ind_collection[i] if train else val_ind_collection[i]
        try:
            s = np.random.choice(list(filter(lambda x:x<num_per_collection[i]-gap-1, data)),avg_num_per_class, replace=False)
            before_ind.append(s)
        except:
            before_ind.append(np.array([]))
    # after-index for each class
    after_ind = [x+gap+1 for x in before_ind]
    # mid-index for each class
    mid_ind = [x+(gap+1)//2 for x in before_ind]
    
    selected_classed = [i for i in range(n_collection) if before_ind[i].shape[0]>0]
    before_imgs = np.concatenate([img_collections[i][before_ind[i]] for i in selected_classed], axis = 0)
    after_imgs = np.concatenate([img_collections[i][after_ind[i]] for i in selected_classed], axis = 0)
    mid_imgs = np.concatenate([img_collections[i][mid_ind[i]] for i in selected_classed], axis = 0)
    
    clipped = np.random.choice(range(before_imgs.shape[0]), batch_size, replace=False)
    before_imgs = before_imgs[clipped]
    mid_imgs = mid_imgs[clipped]
    after_imgs = after_imgs[clipped]
    return (before_imgs, after_imgs), mid_imgs

In [ ]:
def sample_train(batch_size = 8, gap = 1):
    return sample(batch_size)
def sample_dev(batch_size = 8, gap = 1):
    return sample(batch_size, False, gap)